In [1]:
!pip install matplotlib scanpy scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 KB 7.1 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=069e8ca3ece90e1e79addab2f52d87692bcef7a92976

In [2]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.datasets import load_boston, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import numpy as np
import pandas as pd
import jax
import jax.numpy as jnp
from tqdm import tqdm 
import pickle as pkl
from typing import Tuple, Union
import scipy

In [3]:
import scanpy as sc

from warnings import filterwarnings
filterwarnings('ignore')

sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

np.set_printoptions(precision=3)

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.12.2 pynndescent==0.5.8


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1.1 Datasets

In [5]:
!wget https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/sc_training.h5ad

--2023-01-16 16:38:07--  https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/sc_training.h5ad
Resolving saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)... 3.5.129.112
Connecting to saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)|3.5.129.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291266853 (1.2G) [binary/octet-stream]
Saving to: ‘sc_training.h5ad’

sc_training.h5ad    100%[===================>]   1.20G  45.0MB/s    in 29s     

2023-01-16 16:38:37 (42.1 MB/s) - ‘sc_training.h5ad’ saved [1291266853/1291266853]



In [6]:
!wget https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/code/sc_training_visualization.ipynb

--2023-01-16 16:38:37--  https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/code/sc_training_visualization.ipynb
Resolving saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)... 52.219.103.50
Connecting to saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)|52.219.103.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 446576 (436K) [binary/octet-stream]
Saving to: ‘sc_training_visualization.ipynb’

sc_training_visuali 100%[===================>] 436.11K  1.70MB/s    in 0.3s    

2023-01-16 16:38:37 (1.70 MB/s) - ‘sc_training_visualization.ipynb’ saved [446576/446576]



In [7]:
!wget https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/clone_information.csv

--2023-01-16 16:38:37--  https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/clone_information.csv
Resolving saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)... 52.219.103.50
Connecting to saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)|52.219.103.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1542596 (1.5M) [text/csv]
Saving to: ‘clone_information.csv’

clone_information.c 100%[===================>]   1.47M  3.97MB/s    in 0.4s    

2023-01-16 16:38:38 (3.97 MB/s) - ‘clone_information.csv’ saved [1542596/1542596]



In [8]:
!wget https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/guide_abundance.csv

--2023-01-16 16:38:38--  https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/guide_abundance.csv
Resolving saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)... 52.219.103.50
Connecting to saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)|52.219.103.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4452 (4.3K) [text/csv]
Saving to: ‘guide_abundance.csv’

guide_abundance.csv 100%[===================>]   4.35K  --.-KB/s    in 0s      

2023-01-16 16:38:39 (163 MB/s) - ‘guide_abundance.csv’ saved [4452/4452]



In [9]:
!wget https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/scRNA_ATAC.h5

--2023-01-16 16:38:39--  https://saturn-public-data.s3.us-east-2.amazonaws.com/cancer-immunotherapy-challenge/data/scRNA_ATAC.h5
Resolving saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)... 52.219.103.50
Connecting to saturn-public-data.s3.us-east-2.amazonaws.com (saturn-public-data.s3.us-east-2.amazonaws.com)|52.219.103.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87606852 (84M) [application/x-hdf5]
Saving to: ‘scRNA_ATAC.h5’

scRNA_ATAC.h5       100%[===================>]  83.55M  35.2MB/s    in 2.4s    

2023-01-16 16:38:42 (35.2 MB/s) - ‘scRNA_ATAC.h5’ saved [87606852/87606852]



For *read_h5ad()*, see [scanpy documentation](https://scanpy.readthedocs.io/en/stable/generated/scanpy.read_h5ad.html)

In [10]:
adata = sc.read_h5ad('./sc_training.h5ad')
adata

AnnData object with n_obs × n_vars = 28697 × 15077
    obs: 'gRNA_maxID', 'state', 'condition', 'lane'
    layers: 'rawcounts'

For *adata* object type, see [AnnData documentation](https://anndata.readthedocs.io/en/stable/generated/anndata.AnnData.html#anndata.AnnData)

In [11]:
adata.obs

,gRNA_maxID,state,condition,lane
053l1_AAACCTGAGATGTCGG-1,ONE-NON-GENE-SITE-7,terminal exhausted,Unperturbed,lane1
053l1_AAACCTGAGCAACGGT-1,Tox2-3,effector,Tox2,lane1
053l1_AAACCTGAGTACGACG-1,Tpt1-2,effector,Tpt1,lane1
053l1_AAACCTGAGTCGTTTG-1,Tox2-3,terminal exhausted,Tox2,lane1
053l1_AAACCTGAGTGAAGAG-1,Tcf7-2,effector,Tcf7,lane1
...,...,...,...,...
053l4_TTTGTCATCAGGTTCA-1,Tox2-3,other,Tox2,lane4
053l4_TTTGTCATCAGTGTTG-1,Dvl2-3,cycling,Dvl2,lane4
053l4_TTTGTCATCCTCGCAT-1,Zeb2-2,cycling,Zeb2,lane4
053l4_TTTGTCATCTTCAACT-1,Sox4-3,cycling,Sox4,lane4


# 1.1 Preprocess data

In [12]:
expression_data = adata.X
obs = adata.obs
lane_data = obs['lane'].values
lane_data_unique = np.sort(list(set(lane_data))).reshape(-1, 1)

# If the obs['lane'] is categorical variable, it's better to encode it using one-hot encoding 
# before adding it to the X_train to avoid any bias in the model.
encoder = OneHotEncoder()
one_hot_encoded = np.array(encoder.fit_transform(lane_data_unique).toarray())
lane_data_dict = {lane_data_unique[i][0]: one_hot_encoded[i] for i in 
                  range(len(lane_data_unique))}
one_hot_lane_data = [lane_data_dict[lane] for lane in lane_data]

expression_data_sub = expression_data[:10, :10]
expression_data_sub.toarray()

array([[0.512, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ],
       [0.484, 0.484, 0.809, 0.   , 0.   , 0.   , 0.   , 0.   , 0.484,
        0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.694, 0.   , 0.   ,
        0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ],
       [0.   , 1.089, 0.686, 0.686, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ],
       [0.   , 0.193, 0.493, 0.   , 0.   , 0.   , 0.193, 0.   , 0.724,
        0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ],
       [0.292, 0.292, 0.292, 0.292, 0.292, 0.   , 0.   , 0.   , 0.292,
        0.   ],
       [0.412, 0.704, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ]], dtype=float32)

Each row in this matrix represents a cell type (28,697 in total) and the column represents 15,077 genes. The elements indicate the expression levels. 

In [19]:
obs = adata.obs
genes = adata.var.index.tolist()

# embeds one_hot_lane_data into expression data
one_hot_lane_data = np.array(one_hot_lane_data)
from scipy.sparse import hstack
expression_data = hstack((expression_data, one_hot_lane_data))
#expression_data = np.concatenate((expression_data, one_hot_lane_data), axis=1)
#expression_data = np.hstack((expression_data, one_hot_lane_data))

expression_data_csc = expression_data.tocsc()

test_genes = ['Ets1', 'Fosb', 'Mafk', 'Stat3']
validation_genes = ['Aqr', 'Bach2', 'Bhlhe40']

test_indices = [genes.index(x) for x in test_genes if x in genes]
validation_indices = [genes.index(x) for x in validation_genes if x in genes]

X_test = expression_data_csc[:, test_indices]
X_test = X_test.tocsr()
X_val = expression_data_csc[:, validation_indices]
X_val = X_val.tocsr()

all_indices = jnp.arange(expression_data_csc.shape[1])
rm_indices = test_indices + validation_indices
rm_indices.sort()
keep_indices = jnp.setdiff1d(jnp.array(all_indices), jnp.array(rm_indices))

expression_data_csc = expression_data_csc[:, keep_indices]
X_train = expression_data_csc.tocsr()

obs_cond_idx = obs.set_index('condition')

exclude_list = ['Unperturbed', 'Fzd1', 'P2rx7']
dropped_obs = obs_cond_idx.drop(index=exclude_list)

removed_indices = obs_cond_idx.index.isin(exclude_list)

X_train, X_test, X_val = X_train[~removed_indices], X_test[~removed_indices],\
                          X_val[~removed_indices]
X_train, X_test, X_val = X_train.toarray(), X_test.toarray(), X_val.toarray()

Now we have our train, test and validation samples. 



In [20]:
states = ['progenitor', 'effector', 'terminal exhausted', 'cycling', 'other']
ground_truth = {}

for gene in genes:
  state_frequency = {state: 0 for state in states}
  rows = obs[obs['condition'] == gene]

  if not rows.empty:  
    for index, row in rows.iterrows():
        state = row['state']
        if state not in states:
          state_frequency['other'] += 1
        else:
          state_frequency[state] += 1
    
    total_count = sum(state_frequency.values())
    for state, count in state_frequency.items():
        state_frequency[state] = count / total_count
    
    ground_truth[gene] = list(state_frequency.values())

Now we have a dictionary that stores the ground truth, i.e. the cell state frequencies, for each gene. Note that each list in this dictionary sums to 1, as required. We have 64 ground truth vectors because of the 66 knockout experiments that are used for training purposes, 2 did not pass quality control (see: [challenge 1 document](https://drive.google.com/file/d/1rR5oIhETmyVu6Uo5BsWjIokdBZctKzCE/view)). 

Next up, we'll have to generate our y_data and filter the unperturbed, and genes that didn't pass the quality control from our training and testing data.

In [27]:
y_data_dict = ground_truth.copy()
y_data = pd.DataFrame.from_dict(y_data_dict, orient='index', 
                                         columns=['progenitor', 'effector', 
                                         'terminal exhausted', 'cycling', 
                                         'other'])
y_data_expand = pd.DataFrame(columns=y_data.columns, index=dropped_obs.index)
idx = 0
for condition, row in dropped_obs.iterrows():
    gene = condition
    state_data = y_data.loc[gene]
    y_data_expand.iloc[idx] = state_data
    idx += 1

y_train = y_data_expand
y_train

,progenitor,effector,terminal exhausted,cycling,other
condition,,,,,
Tox2,0.017309,0.057004,0.367874,0.53081,0.027002
Tpt1,0.44,0.16,0.12,0.28,0.0
Tox2,0.017309,0.057004,0.367874,0.53081,0.027002
Tcf7,0.104377,0.292929,0.313131,0.276094,0.013468
Tox2,0.017309,0.057004,0.367874,0.53081,0.027002
...,...,...,...,...,...
Tox2,0.017309,0.057004,0.367874,0.53081,0.027002
Dvl2,0.02439,0.115509,0.304188,0.549471,0.006443
Zeb2,0.017483,0.115385,0.282051,0.439394,0.145688


In [28]:
X = X_train
X.shape

(18728, 15074)

In [29]:

obs = obs.reindex(y_train.index)
y_train = np.array(y_train)

y = y_train
y

array([[0.017309023771059313, 0.05700438495268867, 0.36787445188091394,
        0.5308100623124856, 0.027002077082852526],
       [0.44, 0.16, 0.12, 0.28, 0.0],
       [0.017309023771059313, 0.05700438495268867, 0.36787445188091394,
        0.5308100623124856, 0.027002077082852526],
       ...,
       [0.017482517482517484, 0.11538461538461539, 0.28205128205128205,
        0.4393939393939394, 0.1456876456876457],
       [0.0696969696969697, 0.11212121212121212, 0.2909090909090909,
        0.5121212121212121, 0.015151515151515152],
       [0.024390243902439025, 0.1155085135757018, 0.3041877588587207,
        0.5494707777266452, 0.006442705936493327]], dtype=object)

In [31]:
y.shape

(23411, 5)

Note that we do not have y_test and y_val, as these will be used to evaluate the performance of our model in the competition. 

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)

ValueError: ignored

# 2. Random Forest

## 3.1. Random Forest Regression


In [ ]:
rfr = RandomForestRegressor(n_estimators=100, random_state=42, warm_start=True, 
                            n_jobs=-1)

In [ ]:
print('Number of estimators fitted:\n')
for i in tqdm(range(100)):
    # rfr.set_params(n_estimators=i+1)
    rfr.fit(X_train, y_train)

In [ ]:
y_pred = rfr.predict(X_train)
y_pred

In [ ]:
y_pred.shape

In [ ]:
print(mean_absolute_error(y_test, rfr.predict(X_test)).round(3))

___

# 4. Gradient Boosting

In [ ]:
# Loss functions
def MSE(y_true:jnp.array, y_pred:jnp.array):
    return jnp.mean(jnp.sum(jnp.square(y_true-y_pred)))

def CrossEntropy(y_true:jnp.array, y_proba:jnp.array):
    y_proba = jnp.clip(y_proba, 1e-5, 1 - 1e-5)
    return jnp.sum(- y_true * jnp.log(y_proba) - (1 - y_true) * jnp.log(1 - y_proba))

In [ ]:
class GradientBoosting:

    def __init__(self, n_estimators:int=100, learning_rate:float=.1, regression:bool=True, **kwargs):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.regression = regression
        self.loss = MSE if self.regression else CrossEntropy

        self.estimators = []
        for _ in range(self.n_estimators):
                self.estimators.append(DecisionTreeRegressor(**kwargs))

    def fit(self, X:np.array, y:np.array):
        y_pred = np.full(np.shape(y), np.mean(y))
        for i, estimator in enumerate(self.estimators):
            gradient = jax.grad(self.loss, argnums=1)(y.astype(np.float32), y_pred.astype(np.float32))
            self.estimators[i].fit(X, gradient)
            update = self.estimators[i].predict(X)
            y_pred -= (self.learning_rate * update)

    def predict(self, X:np.array):
        y_pred = np.zeros(X.shape[0], dtype=np.float32)
        for estimator in self.estimators:
            y_pred -= (self.learning_rate * estimator.predict(X))
    
        if not self.regression:
            return np.where(1/(1 + np.exp(-y_pred))>.5, 1, 0)
        return y_pred

## 4.1 Gradient Boosting Regressor

In [ ]:
gbr = GradientBoosting(regression=True)

In [ ]:
gbr.fit(X_train_r, y_train_r)

In [ ]:
print(mean_absolute_error(gbr.predict(X_test_r), y_test_r).round(3))

## 4.2 Gradient Boosting Classifier

In [ ]:
gbc = GradientBoosting(regression=False)

In [ ]:
gbc.fit(X_train_c, y_train_c)

In [ ]:
print(accuracy_score(gbc.predict(X_test_c), y_test_c).round(3))